# Importing the necessary libraries

In [ ]:
! pip install --no-index --find-links ../input/reqdata python-gdcm pydicom pylibjpeg-libjpeg

In [4]:
%matplotlib inline
import pandas as pd
import numpy as np
import argparse
import locale

# load and display an image with Matplotlib
from matplotlib import image
from matplotlib import pyplot

# Libraries to help with data visualization
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
import seaborn as sns

## Model Building
import sklearn     
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, auc, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder

## Import Keras & Tensoflow
#from keras.utils import np_utils
import keras
import keras.layers as layers
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,CSVLogger
#from tensorflow.keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalMaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam#from tensorflow.keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalMaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate
from joblib import Parallel, delayed

# Import SMOTE
#from imblearn.over_sampling import SMOTE

import cv2
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import pydicom as pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm.notebook import tqdm as tqdm
import ray
import time

from PIL import Image
import glob

In [5]:
input_dir = "../input/rsna-breast-cancer-detection"
output_dir = "output/train/"

SIZE = 256
EXTENSION = "png"

os.makedirs(output_dir, exist_ok=True)

In [6]:
def process(f, size=256, save_folder="", extension="png"):
    patient = f.split('/')[-2]
    image = f.split('/')[-1][:-4]

    dicom = pydicom.dcmread(f)
    img = dicom.pixel_array

    img = (img - img.min()) / (img.max() - img.min())

    if dicom.PhotometricInterpretation == "MONOCHROME1":
        img = 1 - img

    img = cv2.resize(img, (size, size))

    cv2.imwrite(save_folder + f"{patient}_{image}.{extension}", (img * 255).astype(np.uint8))

# Model Prediction

In [7]:
new_model = tensorflow.keras.models.load_model('/kaggle/input/d/freddeletter/my-model/my_model')

2023-02-07 20:52:41.905469: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [8]:
test_images = glob.glob("/kaggle/input/rsna-breast-cancer-detection/test_images/*/*.dcm")

In [9]:
output_dir_test = "output/test/"
os.makedirs(output_dir_test, exist_ok=True)

In [10]:
_ = Parallel(n_jobs=4)(
    delayed(process)(uid, size=SIZE, save_folder=output_dir_test, extension=EXTENSION)
    for uid in tqdm(test_images)
)

  0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
filenames = list()

def datSetGenerator(path):
    index = 0
    image_list = []
    for filename in glob.glob(path+'*.png'):
        image_list.append(cv2.resize(cv2.imread(filename),(32, 32))/255)
        filenames.append(filename)
        index += 1
        if index == 500000:
            break
    return np.array(image_list)


if __name__ == '__main__':

    # for testing the generator
    path = "output/test/"
    #path = "/home/project/train_images/"
    images_test = datSetGenerator(path)
    print("\n data shape :",images_test.shape)


 data shape : (4, 32, 32, 3)


In [12]:
df_t = pd.read_csv(os.path.join(input_dir, "test.csv"))

In [13]:
df_t

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id
0,2,10008,736471439,L,MLO,81,0,21,10008_L
1,2,10008,1591370361,L,CC,81,0,21,10008_L
2,2,10008,68070693,R,MLO,81,0,21,10008_R
3,2,10008,361203119,R,CC,81,0,21,10008_R


In [14]:
df_tt = df_t.copy()

In [15]:
df_tt

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id
0,2,10008,736471439,L,MLO,81,0,21,10008_L
1,2,10008,1591370361,L,CC,81,0,21,10008_L
2,2,10008,68070693,R,MLO,81,0,21,10008_R
3,2,10008,361203119,R,CC,81,0,21,10008_R


In [16]:
df_tt.drop('prediction_id', axis=1, inplace=True)

In [17]:
## Create dummy variables

def encode_cat_vars(x):
     x = pd.get_dummies(
         x,
         columns=x.select_dtypes(include=["object", "category"]).columns.tolist(),
         drop_first=True,
     )
     return x


ind_vars_num_test = encode_cat_vars(df_tt)
ind_vars_num_test.head()

,site_id,patient_id,image_id,age,implant,machine_id,laterality_R,view_MLO
0,2,10008,736471439,81,0,21,0,1
1,2,10008,1591370361,81,0,21,0,0
2,2,10008,68070693,81,0,21,1,1
3,2,10008,361203119,81,0,21,1,0


In [18]:
#add missing columns view_CC / view_LM / view_LMO / view_ML / view_MLO
ind_vars_num_test["view_CC"] = 0
ind_vars_num_test["view_LM"] = 0
ind_vars_num_test["view_LMO"] = 0
ind_vars_num_test["view_ML"] = 0

In [19]:
ind_vars_num_test

,site_id,patient_id,image_id,age,implant,machine_id,laterality_R,view_MLO,view_CC,view_LM,view_LMO,view_ML
0,2,10008,736471439,81,0,21,0,1,0,0,0,0
1,2,10008,1591370361,81,0,21,0,0,0,0,0,0
2,2,10008,68070693,81,0,21,1,1,0,0,0,0
3,2,10008,361203119,81,0,21,1,0,0,0,0,0


In [20]:
ind_vars_num_test = ind_vars_num_test.filter(['site_id', 'patient_id', 'image_id', 'age', 'implant', 'machine_id', 'laterality_R', 'view_MLO', 'view_CC', 'view_LM','view_LMO','view_ML'])

In [21]:
preds = new_model.predict([ind_vars_num_test, images_test])
df_t['cancer'] = preds

2023-02-07 20:53:12.496985: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [22]:
df_t

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id,cancer
0,2,10008,736471439,L,MLO,81,0,21,10008_L,0.486403
1,2,10008,1591370361,L,CC,81,0,21,10008_L,0.482372
2,2,10008,68070693,R,MLO,81,0,21,10008_R,0.482254
3,2,10008,361203119,R,CC,81,0,21,10008_R,0.479695


In [23]:
output = df_t[['prediction_id','cancer']]

In [27]:
output

,prediction_id,cancer
0,10008_L,0.484388
1,10008_R,0.480974


In [26]:
output = output.groupby('prediction_id', as_index=False).mean()

In [28]:
output.to_csv('submission.csv', index=False)

In [29]:
#os.remove("/kaggle/working/state.db")

import glob
import os
import shutil

location = "/kaggle/working"
dir = 'output'
path = os.path.join(location, dir)
shutil.rmtree(path, ignore_errors = False)